In [5]:
import gym
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation, ResizeObservation
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.env_util import make_vec_env
# --- MUDANÇA 1: Importando DQN em vez de PPO ---
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CheckpointCallback

# A função para criar o ambiente continua exatamente a mesma
def make_env():
    env = gym_super_mario_bros.make('SuperMarioBros-v0')
    env = ResizeObservation(env, shape=84)
    env = GrayScaleObservation(env, keep_dim=True)
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    return env

# A criação do ambiente em paralelo também continua a mesma
vec_env = make_vec_env(make_env, n_envs=4, vec_env_cls=DummyVecEnv)

# O empilhamento de frames também continua o mesmo
env = VecFrameStack(vec_env, n_stack=4)

# O callback para salvar checkpoints também continua o mesmo
checkpoint_callback = CheckpointCallback(save_freq=50000, save_path='./train_DQN/', name_prefix='mario_model_dqn')

# --- MUDANÇA 2: Instanciando o modelo DQN com seus próprios parâmetros ---
# Em vez de PPO, agora usamos DQN.
# DQN usa um 'replay buffer' para aprender com experiências passadas.
model = DQN(
    'CnnPolicy', 
    env, 
    verbose=1, 
    tensorboard_log="./tensorboard_logs_DQN/", # É bom usar uma nova pasta de log
    learning_rate=1e-4, # Taxa de aprendizado
    buffer_size=100000, # Tamanho do "replay buffer" (memória de jogadas)
    learning_starts=1000, # Quantos passos aleatórios dar antes de começar a aprender
    batch_size=32, # Quantas amostras usar para cada atualização do cérebro
    exploration_fraction=0.1, # Fração do treino gasta diminuindo a aleatoriedade
    exploration_final_eps=0.01 # Nível mínimo de aleatoriedade no final
)

# A chamada para iniciar o treinamento é a mesma
model.learn(total_timesteps=2000, callback=checkpoint_callback)

Using cpu device
Wrapping the env in a VecTransposeImage.
Logging to ./tensorboard_logs_DQN/DQN_5


In [12]:
# Importa as bibliotecas necessárias
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

# Cria o ambiente do Super Mario Bros.
# 'SuperMarioBros-v0' geralmente se refere à fase 1-1.
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Simplifica os controles para um conjunto de 7 movimentos essenciais.
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# Loop principal para rodar e visualizar o jogo
done = True
# O loop rodará por 5000 frames. Aumente este número se quiser assistir por mais tempo.
for step in range(5000):
    # Se uma "vida" acabou (done=True), o ambiente é reiniciado.
    if done:
        state = env.reset()
    
    # Pega uma ação aleatória do conjunto de ações possíveis.
    # Isso fará o Mario se mover de forma caótica.
    action = env.action_space.sample()
    
    # Executa a ação no ambiente para avançar um frame.
    state, reward, done, info = env.step(action)
    
    # Renderiza a tela do jogo. É esta linha que abre e atualiza a janela gráfica.
    env.render()

# Fecha a janela e o ambiente ao final do loop.
env.close()